In [4]:
import numpy as np
from types import coroutine

data = np.array([
    [1, 0, 1, 1],  # Pointy, Not Round, Whiskers, Cat
    [0, 1, 0, 0],  # Floppy, Round, No Whiskers, Not Cat
    [1, 1, 1, 1],  # Pointy, Round, Whiskers, Cat
    [0, 0, 0, 0]   # Floppy, Not Round, No Whiskers, Not Cat
])


In [29]:
def entropy(y):
    """
    Computes the entropy of a dataset.

    Entropy is a measure of impurity or disorder in the dataset. It is calculated
    using the formula:
        H(y) = - Σ (p * log2(p)) for all classes in y
    where `p` is the probability of each class in the dataset.

    Args:
        y (ndarray): A 1D NumPy array containing class labels.

    Returns:
        float: The entropy value, representing the impurity of the dataset.

    Steps:
        1. Identify unique class labels and their counts in the dataset.
        2. Compute the probability of each class.
        3. Apply the entropy formula to calculate the impurity.

    """

    unique_classes, counts = np.unique(y, return_counts=True)  # Find unique values and their counts
    probabilities = counts / len(y)  # Calculate probabilities for each class
    entropy_value = -np.sum([p * np.log2(p) for p in probabilities if p > 0])  # Apply the entropy formula
    return entropy_value


In [31]:
def split(data,feature_index,threshold):
  """
     Splits the dataset into two subsets based on a given feature and threshold.

     This function divides the dataset into two parts:
        - The "left" subset contains rows where the feature value is less than
          or equal to the specified threshold.
        - The "right" subset contains rows where the feature value is greater
          than the threshold.

     Args:
        data (ndarray): A NumPy array representing the dataset, where each row
                        is a data sample, and columns are features.
        feature_index (int): The index of the feature to split on.
        threshold (float): The threshold value used for splitting.

     Returns:
        tuple: A tuple containing two NumPy arrays:
            - left (ndarray): The subset of data where the feature value <= threshold.
            - right (ndarray): The subset of data where the feature value > threshold.

     Steps:
        1. Iterate through each row in the dataset.
        2. Compare the value of the specified feature with the threshold.
        3. Assign rows to the left or right subset based on the comparison.



    """
  left=np.array( [ row for row in data if row[feature_index]  <= threshold] )  # Subset where feature <= threshold
  right=np.array( [ row for row in data if row[feature_index] > threshold] ) # Subset where feature > threshold

  return left,right


In [22]:
def best_split():
  """
    Determines the best feature and threshold to split the dataset for a decision tree.

    This function evaluates all possible splits for each feature in the dataset by iterating
    through each feature and its unique values as potential thresholds. It calculates the
    information gain for each split and selects the one that results in the highest information gain.

    Returns:
        best_feature_index (int): Index of the feature that provides the best split.
        best_threshold (float): Threshold value of the best feature for splitting.

    Steps:
        1. Initialize variables to track the best information gain, feature, and threshold.
        2. Loop through each feature in the dataset.
        3. For each feature, determine its unique values as potential split thresholds.
        4. Split the data based on each threshold and calculate the information gain.
        5. Update the best split if the current split results in a higher information gain.
        6. Return the feature index and threshold for the best split.

    Notes:
        - This function assumes the last column in the dataset is the target label.
        - Skips invalid splits where one of the subsets is empty.
    """
  best_gain_information=0
  best_threshold=0
  best_feature_index=0
  x=data[:,:-1]
  y=data[: ,-1]
  num_feature=x.shape[1]
  for feature_index in range(num_feature):
    thresholds=np.unique(data[:,feature_index])
    for threshold in thresholds:
        left,right = split(data,feature_index,threshold)
        if len(left)==0 or len(right)==0 :
          continue

        gain=gain_information(data[: ,-1],left[:, -1],right[: ,-1])

        if gain > best_gain_information:
          best_gain_information = gain
          best_feature_index=feature_index
          best_threshold=threshold

  return best_feature_index , best_threshold

In [32]:
def gain_information(parent,left,right):
    """
    Calculates the information gain from splitting a dataset.

    Information gain measures the reduction in entropy after a dataset is split
    into two subsets. It quantifies how much the uncertainty (impurity) of the
    data is reduced by the split.

    Args:
        parent (ndarray): The labels of the original dataset before splitting.
        left (ndarray): The labels of the subset where the feature value <= threshold.
        right (ndarray): The labels of the subset where the feature value > threshold.

    Returns:
        float: The information gain achieved by splitting the dataset.

    Formula:
        Information Gain = Entropy(parent) -
                           [ (|left| / |parent|) * Entropy(left) +
                             (|right| / |parent|) * Entropy(right) ]

    Steps:
        1. Calculate the proportion of the left and right subsets relative to the parent dataset.
        2. Compute the weighted average of the entropy for the left and right subsets.
        3. Subtract the weighted entropy of the subsets from the parent's entropy to get the information gain.
    """

    num_lfet=len(left)/len(parent)# Proportion of the left subset to the parent dataset
    num_right=len(right)/len(parent)# Proportion of the right subset to the parent dataset
    gain_information=entropy(parent)-((num_lfet*entropy(left))+(num_right*entropy(right)))  # Compute the information gain

    return gain_information



In [33]:
def build_tree(data, depth=0, max_depth=10):
    """
    Builds a decision tree recursively using the input dataset.

    The function splits the dataset into subsets based on the feature and
    threshold that provide the highest information gain. The process continues
    until one of the stopping criteria is met: maximum depth is reached or all
    samples belong to the same class.

    Args:
        data (ndarray): A NumPy array where rows are samples and columns are
                        features, with the last column being the target labels.
        depth (int): The current depth of the tree. Defaults to 0.
        max_depth (int): The maximum depth allowed for the tree. Defaults to 10.

    Returns:
        dict or int: A dictionary representing a node in the decision tree if
                     the tree continues to split, or an integer representing the
                     predicted class if the tree reaches a leaf node.

    Steps:
        1. Split the dataset into features (`X`) and target labels (`y`).
        2. Count the number of samples per class to determine the majority class.
        3. Check stopping criteria:
           - If maximum depth is reached or all labels are identical, return
             the majority class as a leaf node.
        4. Find the best feature and threshold for splitting the data.
        5. Split the data into left and right subsets.
        6. Recursively build the left and right branches of the tree.
        7. Return a node dictionary containing the splitting feature, threshold,
           and pointers to the left and right subtrees.
    """
    X, y = data[:, :-1], data[:, -1]
    num_samples_per_class = [np.sum(y == i) for i in np.unique(y)]
    predicted_class = np.argmax(num_samples_per_class)

    if depth >= max_depth or len(np.unique(y)) == 1:
        return predicted_class

    feature_index, threshold = best_split()
    left, right = split(data, feature_index, threshold)

    node = {
        'feature_index': feature_index,
        'threshold': threshold,
        'left': build_tree(left, depth + 1, max_depth),
        'right': build_tree(right, depth + 1, max_depth)
    }
    return node


In [34]:
def predict(tree, sample):
    """
    Makes a prediction by traversing the decision tree.

    This function recursively traverses the decision tree, starting from the root.
    At each node, it checks the feature value for the sample and decides whether to
    move to the left or right branch based on the threshold. The process continues
    until it reaches a leaf node, which contains the predicted class label.

    Args:
        tree (dict): A dictionary representing a node in the decision tree. If the node is a leaf,
                     it contains the predicted class; otherwise, it contains the feature index,
                     threshold, and the left and right branches (subtrees).
        sample (ndarray): A single data sample (row of features) to be classified. The sample is
                          an array of feature values.

    Returns:
        int: The predicted class label for the input sample.

    Steps:
        1. Check if the current node is a leaf (not a dictionary). If it is, return the class label.
        2. If the current node is not a leaf, compare the sample's feature value with the threshold.
        3. If the sample's feature value is less than or equal to the threshold, recursively call `predict` on the left subtree.
        4. If the sample's feature value is greater than the threshold, recursively call `predict` on the right subtree.
    """
    if not isinstance(tree, dict):
        return tree
    feature_index = tree['feature_index']
    threshold = tree['threshold']

    if sample[feature_index] <= threshold:
        return predict(tree['left'], sample)
    else:
        return predict(tree['right'], sample)


In [35]:
# Build the decision tree
tree = build_tree(data)



In [36]:
# Make a prediction
prediction = predict(tree, [1, 0, 1])  # Example: Pointy, Not Round, Whiskers
print(prediction)


0
